In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from utils.clean import text_to_wordlist
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
from utils.tweaks import reduce_mem_usage

In [2]:
%%time
train = pd.read_csv('data/train_clean.csv')
train = reduce_mem_usage(train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
%%time
test = pd.read_csv('data/test_clean.csv')
test = reduce_mem_usage(test)

In [8]:
train[:5]

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [9]:
test[:5]

,test_id,question1,question2,qid1,qid2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,2447329,3861609
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,2686201,2627256
2,2,What but is the best way to send money from Ch...,What you send money to China?,3479610,3169183
3,3,Which food not emulsifiers?,What foods fibre?,3738446,2245807
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,4059625,320692


In [ ]:
features_list = train.columns.tolist()[6:]
features_list

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train[features_list], train['is_duplicate'], test_size=0.33, random_state=42,stratify=train['is_duplicate'])

In [ ]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'eta': 0.02,
    'max_depth': 4,
    }

d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)

watchlist = [(d_train, 'train'), (d_test, 'test')]

model = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=50, verbose_eval=10)

In [ ]:
xgb.plot_tree(model,num_trees=0)
plt.rcParams['figure.figsize'] = [50, 100]
plt.show()

In [ ]:
xgb.plot_importance(model)
plt.rcParams['figure.figsize'] = [10, 10]
plt.rcParams.update({'font.size': 17})
plt.show()

In [ ]:
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)

In [ ]:
print(classification_report(y_test, clf.predict(X_test)))

In [ ]:
test['is_duplicate'] = test.apply(lambda x: clf.predict(x[features_list]) ,axis=1)

In [ ]:
test[['test_id','is_duplicate']].to_csv('data/1st_submission.csv',index=False)

In [61]:
%%capture --no-display
import keras.layers as layers
from keras.models import Model
from keras import backend as K
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [62]:
%%capture --no-display
import tensorflow as tf
import tensorflow_hub as hub

In [46]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

In [63]:
DROPOUT = 0.1

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

def create_model():
    # Taking the question1 as input and ceating a embedding for each question before feed it to neural network
    q1 = layers.Input(shape=(1,), dtype=tf.string)
    embedding_q1 = layers.Lambda(UniversalEmbedding, output_shape=(512,))(q1)
    # Taking the question2 and doing the same thing mentioned above, using the lambda function
    q2 = layers.Input(shape=(1,), dtype=tf.string)
    embedding_q2 = layers.Lambda(UniversalEmbedding, output_shape=(512,))(q2)

    # Concatenating the both input layer
    merged = layers.concatenate([embedding_q1, embedding_q2])
    merged = layers.Dense(200, activation='relu')(merged)
    merged = layers.Dropout(DROPOUT)(merged)

    # Normalizing the input layer,applying dense and dropout  layer for fully connected model and to avoid overfitting 
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dense(200, activation='relu')(merged)
    merged = layers.Dropout(DROPOUT)(merged)

    merged = layers.BatchNormalization()(merged)
    merged = layers.Dense(200, activation='relu')(merged)
    merged = layers.Dropout(DROPOUT)(merged)

    merged = layers.BatchNormalization()(merged)
    merged = layers.Dense(200, activation='relu')(merged)
    merged = layers.Dropout(DROPOUT)(merged)

    # Using the Sigmoid as the activation function and binary crossentropy for binary classifcation as 0 or 1
    merged = layers.BatchNormalization()(merged)
    pred = layers.Dense(2, activation='sigmoid')(merged)
    model = Model(inputs=[q1,q2], outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

In [37]:
X1 = train['question1']
X2 = train['question2']
y = train['is_duplicate']
# Using the sklearn to split data in question1 and question2 train and test in the ration 80-20 %
X1_train, X1_test,X2_train, X2_test, y_train, y_test = train_test_split(X1, X2, y, test_size=0.2, random_state=42)

train_q1 = X1_train.tolist()
train_q1 = np.array(train_q1, dtype=object)[:, np.newaxis]
train_q2 = X2_train.tolist()
train_q2 = np.array(train_q2, dtype=object)[:, np.newaxis]

train_labels = np.asarray(pd.get_dummies(y_train), dtype = np.int8)

test_q1 = X1_test.tolist()
test_q1 = np.array(test_q1, dtype=object)[:, np.newaxis]
test_q2 = X2_test.tolist()
test_q2 = np.array(test_q2, dtype=object)[:, np.newaxis]

test_labels = np.asarray(pd.get_dummies(y_test), dtype = np.int8)


In [64]:
nn_model = create_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
lambda_10 (Lambda)              (None, 512)          0           input_11[0][0]                   
__________________________________________________________________________________________________
lambda_11 (Lambda)              (None, 512)          0           input_12[0][0]                   
__________________________________________________________________________________________________
concatenat

In [ ]:
from keras.callbacks import ModelCheckpoint

# Creating the tensorflow session to train the model and save checkpoint after every epoch.
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())

    filepath="models/model-{epoch:02d}-{val_acc:.2f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=True, mode='auto', period=1)
    callbacks_list = [checkpoint]
    history = nn_model.fit([train_q1, train_q2], 
            train_labels,
            validation_data=([test_q1, test_q2], test_labels),
            epochs=10,
            batch_size=512, callbacks=callbacks_list)

In [69]:
q1 = input("Type Question 1 here -->")
q2 = input("Type Question 2 here -->") 
q1 = np.array([[q1],[q1]])
q2 = np.array([[q2],[q2]])
# Using the same tensorflow session for embedding the test string
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    # Loading the save weights
    nn_model.load_weights('models/model-01-0.50.hdf5')  
    # Predicting the similarity between the two input questions 
    predicts = nn_model.predict([q1, q2], verbose=0)
    predict_logits = predicts.argmax(axis=1)
    print("----FINAL RESULT----")
    if(predict_logits[0] == 1):
        print("****Questions are Similar****")
    else:
        print("****Questions are not Similar****")

Type Question 1 here --> how are you
Type Question 2 here --> are you alright


----FINAL RESULT----
****Questions are not Similar****
